# Working with OpenAI GPT models

just like any other APIs, you can send a request to openAI chatGPT server and get the response back from your query.

[<img src="https://res.cloudinary.com/practicaldev/image/fetch/s--6UwyTHKO--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/t637s1yazyyxfl31ymmq.jpg">](https://res.cloudinary.com/practicaldev/image/fetch/s--6UwyTHKO--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_800/https://dev-to-uploads.s3.amazonaws.com/uploads/articles/t637s1yazyyxfl31ymmq.jpg)


You need to:
* [Set up an account](https://auth0.openai.com/u/signup/identifier?state=hKFo2SBLZVEyMlJSRDNkbWVMUWVYdU5SVGZKQWltY016ek1POaFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIEJxeTRsb191RnZySEV0b2dlYnRZdGNzQWpZdkRWZjI4o2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q)
* [Get an API Key](https://platform.openai.com/api-keys)
* Add money!

In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
from datasets import load_dataset

from pprint import pprint

# I created a local config.py file to manage my secret keys
from config import OPENAI_KEY 

In [ ]:
API_KEY = OPENAI_KEY

client = OpenAI(api_key=API_KEY)

### Text classification with a LLM

In [ ]:
# download and cache the dataset:
raw_datasets = load_dataset("imdb")

In [ ]:
raw_datasets['train']['text'][1]

In [ ]:
raw_datasets['train']['label'][1]

# Add info on temp and other params

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[],
  temperature=0,
  max_tokens=256
)

In [ ]:
params ={
    "model_name": "gpt-3.5-turbo",
    "temperature": 0.1,
    "max_tokens":256
}

def classifier(input_text, parameters, client=client):
    
    response = client.chat.completions.create(
        model=parameters["model_name"],
        messages=input_text,
        temperature=parameters["temperature"], 
        max_tokens=parameters["max_tokens"],
    )

    return response.choices[0].message.content

In [ ]:
messages=[
    {"role": "system", "content": "You are a useful assitant for the imdb website. You should read the submitted movie review by a user below and decide if it is a positive or negative."},
    {"role": "user", "content": raw_datasets['train']['text'][1]}
  ]

In [ ]:
classifier(messages, params)

I can now run the same query on all the rows and get the response

In [ ]:
data/